In [ ]:
import os
os.chdir('../')
print(os.getcwd())

In [ ]:
import torch 
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

In [ ]:
# Device configuration
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 2
num_classes = 2
batch_size = 1
learning_rate = 0.00001

In [ ]:
from src.dataset import multi_task_dataset

In [ ]:
df = pd.read_csv('input/data.csv')

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df.shape

In [ ]:
df_train = df.loc[(df.Type=='train') & (df.Subject==4)].reset_index(drop = True)
df_test = df.loc[(df.Type=='test') & (df.Subject==4)].reset_index(drop = True)
df_train.shape, df_test.shape

In [ ]:
train_dataset = multi_task_dataset(df_train)
test_dataset = multi_task_dataset(df_test)

In [ ]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [ ]:
def conv_block(in_channels, out_channels):
    '''Convolution Block of 3x3 kernels + batch norm + maxpool of 2x2'''

    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 5, padding=2),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
def set_grad(model, grad):
    for param in model.parameters():
#         print(param.shape)
        param.requires_grad = grad

In [ ]:
class multi_task_model(nn.Module):
    def __init__(self):
        super(multi_task_model, self).__init__()

        self.conv1 = conv_block(1, 20)
        self.conv2 = conv_block(20, 40)
        self.conv3 = conv_block(40, 60)        
        self.fc =  nn.Linear(9 * 4 * 60, 100 )    
        
        #heads
        self.y1o = nn.Linear(100,2)
        self.y2o = nn.Linear(100,2)
        self.y3o = nn.Linear(100,2)
        self.y4o = nn.Linear(100, 2)
        self.y5o = nn.Linear(100, 2)
        self.y6o = nn.Linear(100,2)
        self.y7o = nn.Linear(100, 2)
        self.y8o = nn.Linear(100, 2)
        self.y9o = nn.Linear(100, 2)
        
    def freeze(self, subject):
        set_grad(self.y1o, False)
        set_grad(self.y2o, False)
        set_grad(self.y3o, False)
        set_grad(self.y4o, False)
        set_grad(self.y5o, False)
        set_grad(self.y6o, False)
        set_grad(self.y7o, False)
        set_grad(self.y8o, False)
        set_grad(self.y9o, False)
        
        if subject==1:
            set_grad(self.y1o, True)
        if subject==2:
            set_grad(self.y2o, True)
        if subject==3:
            set_grad(self.y3o, True)
        if subject==4:
            set_grad(self.y4o, True)
        if subject==5:
            set_grad(self.y5o, True)
        if subject==6:
            set_grad(self.y6o, True)
        if subject==7:
            set_grad(self.y7o, True)
        if subject==8:
            set_grad(self.y8o, True)
        if subject==9:
            set_grad(self.y9o, True)
            

    def forward(self, x): 
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
#         print('head shape ',x.shape)
        x = x.reshape(x.size(0), -1)
        x1 = self.fc(x)
        
        # heads
        out1 = self.y1o(x1)
        out2 = self.y2o(x1)
        out3 = self.y3o(x1)
        out4 = self.y4o(x1)
        out5 = self.y5o(x1)
        out6 = self.y6o(x1)
        out7 = self.y7o(x1)
        out8 = self.y8o(x1)
        out9 = self.y9o(x1)
                
        #y5o = torch.sigmoid(self.y5o(x1)) #should be sigmoid
#         print('Losses = ',y4o, '\n')
        return out1, out2, out3, out4, out5, out6, out7, out8, out9

In [ ]:
model = multi_task_model()
model = model.to(device)


In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels, subjects) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        subjects = subjects.to(device)
        outputs = model(images)
        loss = criterion(outputs[subjects-1], labels)
         
#         model.freeze(subjects)
#         model.to(device)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 2000 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [ ]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for image, label, subject in test_loader:
        image = image.to(device)
        label = label.to(device)
        outputs = model(image)
        _, predicted = torch.max(outputs[subject-1].data, 1)
#         print('predicted and label = ', predicted.item(), label.item())     
        
        total += label.size(0)
        correct += (predicted == label).sum().item()
#         print(predicted, label.item())
# print(total, correct)
print('Test Accuracy : {} %'.format(100 * correct / total))

# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for image, label, subject in train_loader:
        image = image.to(device)
        label = label.to(device)
        outputs = model(image)
        _, predicted = torch.max(outputs[subject-1].data, 1)
#         print('predicted and label = ', predicted.item(), label.item())     
        
        total += label.size(0)
        correct += (predicted == label).sum().item()
#         print(predicted, label.item())
# print(total, correct)
print('Train Accuracy : {} %'.format(100 * correct / total))